# 순환신경망 (RNN)
 - 같은 요소가 순서지어 반복해서 나타나는 데이터
  - 소리 
  - 자연어 
  - 동영상 

![img1](img/1.PNG)
 - 영향이 여러곳으로 갈 수 있다는 부분이다.
 - 안녕하세요가 안이 뒤쪽으로 가면서도 영향을 미치고 녕도 뒤로가면서 영향을 끼치는 모냥

![img2](img/2.PNG)

## 실습
 - 컴퓨터에게 2자리 수 덧셈을 시킬 것이다. 
 - 47 + 21 = 68 

In [1]:
import random

from keras.layers import Activation, Dense, LSTM, RepeatVector
from keras.layers.wrappers import TimeDistributed
from keras.models import Sequential
import numpy

Using TensorFlow backend.


In [2]:
DIGITS = 2 # 2자리 숫자라서 Digits 2
MAX_LEN = 2 * DIGITS + 1 # 47 + 21 5자리 최대.
NUM_CHAR = len('0123456789+') # 나올 수 있는 글자의 종류 

In [3]:
MAX_LEN

5

In [4]:
NUM_CHAR

11

In [5]:
N = 10000

In [6]:
x = numpy.zeros((N, MAX_LEN, NUM_CHAR), dtype=numpy.bool)
y = numpy.zeros((N, DIGITS + 1, NUM_CHAR), dtype=numpy.bool)

 - 덧셈을 one-hot encoding하기 위해서 
 - 47 + 21 의 경우 
  - 0 0 0 0 1 0 0 0 0 0 0 
  - 0 0 0 0 0 0 0 1 0 0 0
  - 0 0 0 0 0 0 0 0 0 0 1
  - 0 0 1 0 0 0 0 0 0 0 0 
  - 0 1 0 0 0 0 0 0 0 0 0  
 - 위와 같은 형태로 변환 

In [7]:
x

array([[[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False]],

       [[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False]],

       [[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False]],

       ..., 
       [[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [Fal

In [8]:
y

array([[[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False]],

       [[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False]],

       [[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False]],

       ..., 
       [[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False]],

       [[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False]],

       [[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        

In [9]:
for i in range(N):
    a = random.randint(1, 99)
    b = random.randint(1, 99)
    x[i, 0, a // 10] = True # 몫
    x[i, 1, a % 10] = True # 나머지
    x[i, 2, 10] = True # + 자리에 True
    x[i, 3, b // 10] = True # 몫
    x[i, 4, b % 10] = True # 나머지.
    
    y[i, 0, (a + b) // 100] = True # 실제 정답
    y[i, 1, (a + b) % 100 // 10] = True # 몫
    y[i, 2, (a + b) % 10] = True # 나머지

### 번역기를 만든다면.
 - 위의 숫자 자리에 단어라고 생각하고 TDM형태로 만들면 될 것이다. 

In [11]:
HIDDEN = 128 #  LSM의 Hidden을 몇개를 할 것인지?

In [12]:
model = Sequential()
model.add(LSTM(HIDDEN, input_shape=(MAX_LEN, NUM_CHAR)))
model.add(RepeatVector(DIGITS + 1))
model.add(LSTM(HIDDEN, activation='relu', return_sequences=True)) # return_sequenceses 가 ture가 ㅇ니면 답이 안나간다 1자리만 나간다 
model.add(LSTM(HIDDEN, activation='relu', return_sequences=True))
model.add(TimeDistributed(Dense(NUM_CHAR)))
model.add(Activation('softmax'))

In [13]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128)               71680     
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 3, 128)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 3, 128)            131584    
_________________________________________________________________
lstm_3 (LSTM)                (None, 3, 128)            131584    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 3, 11)             1419      
_________________________________________________________________
activation_1 (Activation)    (None, 3, 11)             0         
Total params: 336,267.0
Trainable params: 336,267.0
Non-trainable params: 0.0
________________________________________________________________

 - MNIST처럼 맞고 틀리고를 봐야하므로 categorical_crossenytopy로 사용.

In [14]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [15]:
model.fit(x, y, batch_size=100, epochs=50, verbose=1)

Epoch 1/50
10000/10000 [==============================] - 9s - loss: 1.9904 - acc: 0.2489     
Epoch 2/50
10000/10000 [==============================] - 7s - loss: 1.6992 - acc: 0.3275     
Epoch 3/50
10000/10000 [==============================] - 8s - loss: 1.6531 - acc: 0.3543     
Epoch 4/50
10000/10000 [==============================] - 8s - loss: 1.5132 - acc: 0.4060     
Epoch 5/50
10000/10000 [==============================] - 8s - loss: 1.2216 - acc: 0.5113     
Epoch 6/50
10000/10000 [==============================] - 7s - loss: 1.0797 - acc: 0.5714     
Epoch 7/50
10000/10000 [==============================] - 7s - loss: 0.9758 - acc: 0.6177     
Epoch 8/50
10000/10000 [==============================] - 7s - loss: 0.9514 - acc: 0.6270     
Epoch 9/50
10000/10000 [==============================] - 7s - loss: 0.9354 - acc: 0.6347     
Epoch 10/50
10000/10000 [==============================] - 7s - loss: 0.9101 - acc: 0.6435     
Epoch 11/50
10000/10000 [========================

In [18]:
model.fit(x, y, batch_size=100, epochs=20, verbose=1)

Epoch 1/20
10000/10000 [==============================] - 7s - loss: 0.0102 - acc: 0.9987     
Epoch 2/20
10000/10000 [==============================] - 7s - loss: 0.0090 - acc: 0.9985     
Epoch 3/20
10000/10000 [==============================] - 7s - loss: 0.0070 - acc: 0.9988     
Epoch 4/20
10000/10000 [==============================] - 7s - loss: 0.1176 - acc: 0.9651     
Epoch 5/20
10000/10000 [==============================] - 7s - loss: 0.0250 - acc: 0.9953     
Epoch 6/20
10000/10000 [==============================] - 7s - loss: 0.0104 - acc: 0.9986     
Epoch 7/20
10000/10000 [==============================] - 7s - loss: 0.0079 - acc: 0.9990     
Epoch 8/20
10000/10000 [==============================] - 7s - loss: 0.0060 - acc: 0.9991     
Epoch 9/20
10000/10000 [==============================] - 7s - loss: 0.0084 - acc: 0.9980     
Epoch 10/20
10000/10000 [==============================] - 7s - loss: 0.0073 - acc: 0.9985     
Epoch 11/20
10000/10000 [========================

In [19]:
p = model.predict(x[0:20])

In [20]:
for i in range(20):
    print('x', numpy.argmax(x[i, 0]), numpy.argmax(x[i, 1]), numpy.argmax(x[i, 3]), numpy.argmax(x[i, 4]))
    print('y', numpy.argmax(y[i, 0]), numpy.argmax(y[i, 1]), numpy.argmax(y[i, 2]))
    print('p', numpy.argmax(p[i, 0]), numpy.argmax(p[i, 1]), numpy.argmax(p[i, 2]))
    print('-' * 30)

x 8 9 4 0
y 1 2 9
p 1 2 9
------------------------------
x 5 3 8 8
y 1 4 1
p 1 4 1
------------------------------
x 2 2 8 8
y 1 1 0
p 1 1 0
------------------------------
x 4 1 9 0
y 1 3 1
p 1 3 1
------------------------------
x 6 2 8 0
y 1 4 2
p 1 4 2
------------------------------
x 7 0 3 5
y 1 0 5
p 1 0 5
------------------------------
x 4 7 3 8
y 0 8 5
p 0 8 5
------------------------------
x 8 3 9 1
y 1 7 4
p 1 7 4
------------------------------
x 9 5 7 1
y 1 6 6
p 1 6 6
------------------------------
x 2 2 3 2
y 0 5 4
p 0 5 4
------------------------------
x 8 7 5 1
y 1 3 8
p 1 3 8
------------------------------
x 1 3 3 4
y 0 4 7
p 0 4 7
------------------------------
x 0 9 1 3
y 0 2 2
p 0 2 2
------------------------------
x 7 8 6 8
y 1 4 6
p 1 4 6
------------------------------
x 1 3 5 5
y 0 6 8
p 0 6 8
------------------------------
x 6 4 0 7
y 0 7 1
p 0 7 1
------------------------------
x 5 6 9 3
y 1 4 9
p 1 4 9
------------------------------
x 8 0 0 3
y 0 8 3
p 0 8 3
-----